In [ ]:
def recommend_and_reason_sentence(title, idx_map, df, sim_matrix):
    try:
        idx = idx_map[title]
    except:
        return '', ''

    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]

    recommendations = []
    reason_sentences = []

    input_genres = set(df.loc[idx, 'genres'].replace(' ', '').split(','))
    input_keywords = set(str(df.loc[idx, 'keywords']).lower().split(', '))

    for i, _ in sim_scores:
        row = df.iloc[i]
        rec_title = row['movieNm']
        rec_genres = set(row['genres'].replace(' ', '').split(','))
        rec_keywords = set(str(row['keywords']).lower().split(', '))

        # 교집합
        genre_overlap = input_genres & rec_genres
        keyword_overlap = input_keywords & rec_keywords

        # 자연어 형태의 추천 이유 문장 만들기
        reason_parts = []
        if genre_overlap:
            genre_text = '’, ‘'.join(genre_overlap)
            reason_parts.append(f"‘{genre_text}’ 장르를 공유하며")
        if keyword_overlap:
            keyword_text = '’, ‘'.join(keyword_overlap)
            reason_parts.append(f"키워드 ‘{keyword_text}’가 유사합니다")

        if reason_parts:
            reason_sentence = f"{rec_title}는 이 영화와 " + ', '.join(reason_parts) + '.'
        else:
            reason_sentence = f"{rec_title}는 유사한 영화입니다."

        recommendations.append(rec_title)
        reason_sentences.append(reason_sentence)

    return ', '.join(recommendations), ' '.join(reason_sentences)

In [ ]:
title_to_idx = pd.Series(movies_df.index, index=movies_df['movieNm']).to_dict()

In [ ]:
recommendations = []
reasons = []

for title in movies_df['movieNm']:
    recs, reason_text = recommend_and_reason_sentence(title, title_to_idx, movies_df, cosine_sim)
    recommendations.append(recs)
    reasons.append(reason_text)

movies_df['recommend'] = recommendations
movies_df['reason'] = reasons

In [ ]:
movies_df.to_csv('movie_with_recommendation_and_reason.csv', index=False, encoding='utf-8-sig')